In [10]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [11]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: gensim in c:\users\ankit dash\appdata\local\programs\python\python311\lib\site-packages (4.3.1)




[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
pip install transformers


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# Define a function to create embeddings for each tweet
def create_avg_embeddings(tweet):
    # Tokenize the tweet and convert tokens to IDs
    inputs = tokenizer.encode_plus(tweet, add_special_tokens=True, return_tensors='pt')

    # Pass the input IDs through the model to get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()

    # Convert embeddings to a numpy array and return it
    return embeddings.mean(dim=0).numpy()

In [16]:
import os
import torch

In [36]:
# Define the path to the folder containing the text files
data_folder = 'data/'

# Initialize the list for average embeddings
avg_embeddings = []

# Loop through each text file in the data folder and create average embeddings
for file_name in os.listdir(data_folder):
    file_path = os.path.join(data_folder, file_name)
    if not os.path.isfile(file_path):
        continue

    # Create average embeddings for the current text file
    current_avg_embeddings = create_avg_embeddings(file_path)
    
    # Convert current_avg_embeddings from NumPy array to list
    current_avg_embeddings = current_avg_embeddings.tolist()
    
    # Append the current average embeddings to the list
    avg_embeddings.append(current_avg_embeddings)

# Convert avg_embeddi

# Print the type of avg_embeddings (should be list)
print((avg_embeddings))

[-0.014243022538721561, -0.01344191562384367, -0.014862656593322754]


In [18]:
import csv
import pandas as pd

In [37]:
import os

# Loop through all the CSV files in the directory
directory = '../../go/files/'
output_directory = 'embeddings'  # Output directory for saving the files

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Read in the tweets from the file
        filepath = os.path.join(directory, filename)
        # Create a DataFrame from the CSV file
        df = pd.read_csv(filepath)
        # Create embeddings for each tweet and store in a list
        embeddings = [create_avg_embeddings(tweet) for tweet in df['Tweet']]
        df['embedding'] = embeddings
        
        # Save the DataFrame with the new column as a new CSV file
        output_filename = 'embeddings_' + filename
        output_filepath = os.path.join(output_directory, output_filename)
        df.to_csv(output_filepath, index=False)

In [48]:
import os
import pandas as pd

# Define the directory path containing the CSV files
directory = 'embeddings/'

# Loop through all the CSV files in the directory
for i, filename in enumerate(os.listdir(directory)):
    if filename.endswith('.csv'):
        # Read the CSV file
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)

        # Get the current element from the list
        current_element = avg_embeddings[i]

        # Initialize the list for embedding differences
        embedding_diff = []

        # Calculate the absolute difference for each tweet in the current file
        for tweet_embedding in df['embedding']:
            embedding_diff.append(abs(tweet_embedding - current_element))
        print(embedding_diff)

[0.0005849125387215621, 0.00023890053872156064, 0.00010535453872156063, 0.0004030025387215617, 0.0007068850387215615, 0.002155595538721562, 0.0003444215387215614, 0.000940928538721561, 0.0012031355387215614, 0.0013837185387215612, 0.0016804025387215614, 0.0019043815387215623, 0.0008801705387215617, 0.0021950345387215606]
[0.0003334376238436701, 1.7657876156330563e-05, 5.410337615632983e-05, 0.0006292036238436695, 0.0009243303761563293, 0.00118818362384367, 0.00041539337615633005, 0.0003875796238436702, 0.0002478786238436694, 0.00044789762384367035, 0.0008986756238436693, 0.0009983051238436694, 0.0008504296238436692, 0.0025017886238436707]
[0.0013271595933227534, 0.0028388725933227547, 0.0016235305933227535, 0.0013576445933227536, 0.0009864045933227535, 0.0018577735933227538, 0.0009277875933227531, 0.0003962825933227533, 0.0018887145933227534, 0.0011482210933227537, 0.0011430185933227546, 0.0015671505933227536, 0.0015116805933227534, 0.001956625593322754]
